In [1]:
import pickle
import cv2
import os
import json
import csv
import numpy as np

In [2]:
with open('../../temp/oi_600_hierarchy.pkl', 'rb') as fin:
    hierarchy = pickle.load(fin)

In [3]:
with open('../../../wsod/metadata/ont_m18/180_classes.csv', 'r') as fin:
    labels_filter = [item.strip() for item in fin]

In [57]:
with open('../../../wsod/metadata/ont_m18/mapping.pkl', 'rb') as fin:
    mid2ont, syn2mid, single_mids, mid2syn, class2ont, ont2name, class_names = pickle.load(fin)

In [5]:
with open('../../temp/oi_coco_label_names.pkl', 'rb') as fin:
    oi_coco_mid2name = pickle.load(fin)

In [6]:
mid2name = {}
for mid in oi_coco_mid2name:
    mid2name[mid] = oi_coco_mid2name[mid]
for mid in class_names:
    mid2name[mid] = class_names[mid]
    

In [7]:
with open('../../results/det_results_merged_25.pkl', 'rb') as fin:
    det_results = pickle.load(fin)

In [50]:
label_map = {}
label_map_hierarchy = {}

for key in mid2name:
    label = key
    if label in mid2syn:
        label = mid2syn[label]
    label_hierarchy = [label]
    print(f'looking for {mid2name.get(label)} ({label})... ', end='')
    while label not in labels_filter:
        label = hierarchy.get(label)
        if label == None: 
            break
        if label in mid2syn:
            label = mid2syn[label]
        label_hierarchy.append(label)
        print(f'changed to {mid2name.get(label)} ({label}); ', end='')
    if label == None:
        print(f'NOT found.')
        continue
    print(f'found.')
    label_map[key] = label
    label_map_hierarchy[key] = list(reversed(label_hierarchy[:-1]))

looking for Toilet (/m/09g1w)... changed to Plumbing fixture (/m/02pkr5); changed to None (/j/cjpcdr); changed to None (/m/0bl9f); NOT found.
looking for Clock (/m/01x3z)... changed to None (/m/0h8nzzj); changed to None (/j/cjpcdr); changed to None (/m/0bl9f); NOT found.
looking for toaster (/m/01k6s3)... changed to Home appliance (/m/019dx1); changed to None (/j/cjpcdr); changed to None (/m/0bl9f); NOT found.
looking for Laptop (/m/01c648)... changed to COM (Electronic device) (/m/0bs7_0t); found.
looking for VEH.WheeledVehicle (Bicycle) (/m/0199g)... found.
looking for Chair (/m/01mzpv)... changed to Furniture (/m/0c_jw); changed to None (/j/cjpcdr); changed to None (/m/0bl9f); NOT found.
looking for Pizza (/m/0663v)... changed to Food (/m/02wbm); changed to None (/m/0bl9f); NOT found.
looking for frisbee (/m/02wmf)... changed to Toy (/m/0138tl); changed to None (/j/cjpcdr); changed to None (/m/0bl9f); NOT found.
looking for Bear (/m/01dws)... changed to Carnivore (/m/01lrl); changed

In [37]:
oi_coco_mid2name['/m/011q46kg'] = 'Container'

In [58]:
mid2name_all = {}
for mid in label_map:
    if mid in class_names:
        mid2name_all[mid] = class_names[mid]
    else:
        c, s = class_names[label_map[mid]].split(' (')
        s = s[:-1].replace(' ', '_')
        fr = '.'.join([s] + [oi_coco_mid2name[m].replace(' ', '_') for m in label_map_hierarchy[mid]])
        c = c + f' ({fr})'
        mid2name_all[mid] = c

In [61]:
len(mid2name_all)

232

In [45]:
label_map['/m/01c648']

'/m/0bs7_0t'

In [48]:
label_map_hierarchy['/m/01c648']

['/m/0bs7_0t', '/m/01c648']

In [46]:
len(oi_coco_mid2name)

551

In [49]:
class_names['/m/0bs7_0t']

'COM (Electronic device)'

In [59]:
mid2name_all

{'/m/01c648': 'COM (Electronic_device.Laptop)',
 '/m/0199g': 'VEH.WheeledVehicle (Bicycle)',
 '/m/0k4j': 'VEH.WheeledVehicle.Car (Car)',
 '/m/019jd': 'VEH.Watercraft.Boat (Boat)',
 '/m/07r04': 'VEH.WheeledVehicle.Truck (Truck)',
 '/m/07c52': 'COM (Electronic_device.Television)',
 '/m/04ctx': 'WEA.DaggerKnifeSword (Knife)',
 '/m/0cmf2': 'VEH.Aircraft.Airplane (Airplane)',
 '/m/01g317': 'PER (Person)',
 '/m/01lsmm': 'COM.Equipment (Tool.Scissors)',
 '/m/04dr76w': 'COM.Equipment (Tool.Container.Bottle)',
 '/m/01s55n': 'COM (Luggage_and_bags.Suitcase)',
 '/m/0qjjc': 'COM (Electronic_device.Remote_control)',
 '/m/050k8': 'COM (Telephone.Mobile_phone)',
 '/m/080hkjn': 'COM (Luggage_and_bags.Handbag)',
 '/m/07jdr': 'VEH.WheeledVehicle.Train (Train)',
 '/m/01m2v': 'COM (Electronic_device.Computer_keyboard)',
 '/m/01bjv': 'VEH.WheeledVehicle.Bus (Bus)',
 '/m/04_sv': 'VEH.WheeledVehicle (Motorcycle)',
 '/m/01940j': 'COM (Luggage_and_bags.Backpack)',
 '/m/020lf': 'COM (Electronic_device.Mouse)',


In [60]:
with open('../../../wsod/metadata/ont_m18/class_names_all.pkl', 'wb') as fout:
    pickle.dump(mid2name_all, fout)

In [9]:
det_results_filtered = {}
for key, val in det_results.items():
    det_results_filtered[key] = []
    for det in val:
        label = label_map.get(det['label'])
        if label == None:
            continue
        det_results_filtered[key].append({
            'label': label,
            'score': det['score'],
            'bbox': det['bbox'],
            'bbox_normalized': det['bbox_normalized'],
            'model': det['model'],            
        })


In [12]:
with open('../../results/det_results_postproc_25.pkl', 'wb') as fout:
    pickle.dump(det_results_filtered, fout)

## Sorting the results

In [13]:
det_results_sorted = {}
for imgid, det in det_results_filtered.items():
    detection_classes = [item['label'] for item in det]
    detection_scores = [item['score'] for item in det]
    detection_boxes = [np.asarray(item['bbox']) for item in det]
    detection_boxes_n = [np.asarray(item['bbox_normalized']) for item in det]
    detection_models = [item['model'] for item in det]

    sort_idx = np.argsort(-np.asarray(detection_scores))
    
    detection_classes = [detection_classes[i] for i in sort_idx]
    detection_scores = [detection_scores[i] for i in sort_idx]
    detection_boxes = [detection_boxes[i] for i in sort_idx]
    detection_boxes_n = [detection_boxes_n[i] for i in sort_idx]
    detection_models = [detection_models[i] for i in sort_idx]
    
    det_results_sorted[imgid] = {
        'detection_classes': detection_classes,
        'detection_scores': detection_scores,
        'detection_boxes': detection_boxes,
        'detection_boxes_normalized': detection_boxes_n,
        'detection_models': detection_models,            
        'num_detections': len(detection_classes)
    }
    

In [14]:
with open('../../results/det_results_postproc_25_sorted.pkl', 'wb') as fout:
    pickle.dump(det_results_sorted, fout)